<a href="https://colab.research.google.com/github/cleucio-diogo/planejamento/blob/main/analise_bases_parciais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm  # Importe a biblioteca tqdm
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.subplots as sp  # Importe o módulo subplots
import seaborn as sns
import plotly.express as px
import os

from google.colab import drive
raiz = '/content/drive'
drive.mount(raiz)

Mounted at /content/drive


In [ ]:
##### BLOCO DE FUNÇÕES AUXILIARES ####################

file_path = "/content/drive/MyDrive/csv_files/"
input_path = file_path+'input/'
output_path = file_path+'output/'

import hashlib

def gerar_identificador_unico(texto, comprimento=6):
    """
    Gera um identificador único de até 'comprimento' caracteres a partir de uma string.

    Parâmetros:
    texto (str): A string de entrada para gerar o identificador.
    comprimento (int): O número de caracteres desejados para o identificador (padrão é 6).

    Retorna:
    str: Um identificador único com o número de caracteres especificado.
    """
    contador = 0
    while True:
        # Gerar um hash SHA-256 do texto concatenado com o contador
        hash_object = hashlib.sha256((texto + str(contador)).encode())
        # Converter o hash para hexadecimal
        hash_hex = hash_object.hexdigest()
        # Retornar os primeiros 'comprimento' caracteres do hash
        identificador = hash_hex[:comprimento]

        # Retornar o identificador
        return identificador

        contador += 1  # Incrementar o contador para a próxima tentativa

def export_to_csv(dataframe, file_path,file_name=None, index=False, sep=';', encoding='utf-8'):
    """
    Exporta um DataFrame para um arquivo CSV.

    Parameters:
    - dataframe: DataFrame a ser exportado.
    - file_path: Caminho do arquivo onde o CSV será salvo. Se não fornecido, usa o nome do DataFrame.
    - index: Se True, inclui o índice do DataFrame no CSV.
    - sep: Separador a ser usado no CSV.
    - encoding: Codificação a ser usada no CSV.
    """
    if file_name is None:
        # Usar o nome do DataFrame como o nome do arquivo
        dataframe_name = [name for name, var in globals().items() if var is dataframe][0]
        file_name = f"{dataframe_name}.csv"

        # Criar o diretório se ele não existir
        os.makedirs(file_path, exist_ok=True)

        # Montar o caminho completo do arquivo
        full_path = os.path.join(file_path, file_name)
    else:
        full_path = file_path + file_name
    dataframe.to_csv(full_path, index=index, sep=sep, encoding=encoding)

def limpar_colunas(df, colunas):
  for col in colunas:
    #() => permite quebras de linhas quando você quiser, sem afetar o código. Isso é ótimo para inserir comentários em pedaços de código que deveriam ficar em uma única linha
    df[col] = (
      df[col].apply(str).str.lower() #padronizando e transformando tudo em minusculas;
      .str.replace('\n','') #removendo quebras de linhas
      .str.replace('r$','') #removendo r$
      .str.replace('.', '', regex=False) #removendo separadores de milhar
      .str.replace(',', '.', regex=False) # trocando o separador de dezena
      .str.replace('nan','')
      .str.strip() #removendo espaços em branco
    )

def to_numeric(df, colunas):
  for col in colunas:
    df[col] = (
        pd.to_numeric(df[col], errors='coerce').astype(float)
    ).fillna(0)

def criar_faixas_valor(df, colunas, metodo='sturges', implace = True):
  """
  Cria faixas de valor para colunas específicas em um DataFrame.

  Parâmetros:
  df (pd.DataFrame): DataFrame contendo os dados.
  colunas (list): Lista de nomes de colunas para criar as faixas de valor.
  metodo (str): Método para determinar o número de classes ('sturges' ou 'sqrt').

  Retorna:
  implace= True -> pd.DataFrame: dataframe original com as novas colunas de faixas de valor.
  implace= False -> pd.DataFrame: inclui no dataframe original as novas colunas de faixas de valor.
  """
  for col in colunas:
    # Obter os valores da coluna
    valores = df[col]

    # 1. Calcular a amplitude total
    L_min = valores.min()
    L_max = valores.max()
    AT = L_max - L_min

    # 2. Determinar o número de classes
    n = len(valores)
    if metodo == 'sturges':
        k = int(1 + 3.322 * np.log10(n))  # Regra de Sturges
    elif metodo == 'sqrt':
        k = int(np.sqrt(n))  # Regra da Raiz Quadrada
    else:
        raise ValueError("Método deve ser 'sturges' ou 'sqrt'.")

    # 3. Calcular a amplitude de cada classe
    h = AT / k

    # 4. Definir os limites das classes
    faixas = []
    for i in range(k):
        lower_limit = L_min + i * h
        upper_limit = lower_limit + h
        faixa = f"{lower_limit:.2f} - {upper_limit:.2f}"
        faixas.append(faixa)

    # Adicionar a nova coluna de faixas ao DataFrame
    df[f"{col}_faixas"] = pd.cut(df[col], bins=[-np.inf] + [float(limite.split('-')[0].strip()) for limite in faixas[:-1]] + [np.inf], labels=faixas)
  if not implace: return df

In [ ]:
#importando as faixas
colunas = ['FASE', 'FAIXA_BANCO', 'FAIXA_FIM']

#encoding='utf-8-sig #se parar de funcionar tentar esse charset'
faixas_df = pd.read_csv(
    input_path + 'dFaixas.csv',
    encoding='latin-1',
    sep=';',
    low_memory=False,
    dtype = str,
    usecols = colunas
  )
faixas_df.columns

Index(['FASE', 'FAIXA_BANCO', 'FAIXA_FIM'], dtype='object')

In [ ]:
#importando e tratando a base dos meses
base_columns = [
    'CTT_OP', 'GRUPO_PERF', 'CARTEIRA','MARCACAO', 'FASE', 'UF', 'ATRASO_INI', 'FX_ATRASO_INI', 'VLCLSER',
    'VL_LIQ_VENDA','TIPO_SUCESSO', 'DATA_PAGAMENTO', 'CAIXA_PERF', 'DT_ATRASO_INI','META_CC', 'META_RET', 'DATA_REF'
]

base_df = pd.read_csv(
    input_path + 'bases.csv',
    encoding='latin-1',
    sep=';',
    usecols = base_columns,
    low_memory=False
)

base_df.drop_duplicates(inplace=True) #excluindo duplicatas

# adicionando a faixa fim, do arquivo dfaixas.csv
base_df = pd.merge(
    base_df,
    faixas_df,
    left_on=['FASE', 'FX_ATRASO_INI'],
    right_on=['FASE', 'FAIXA_BANCO'],
    how='left',
)

#converte data numerica para o formato yyyy-mm-dd e depois para dd/mm/yyyy
base_df['DATA_PAGAMENTO'] = pd.to_datetime(base_df['DATA_PAGAMENTO'], unit='D', origin='1899-12-30').dt.strftime('%d/%m/%Y')

#adicionando a coluna de ANOMES para poder dar merge com df_sucessos
base_df['DATA_REF'] = pd.to_datetime(base_df['DATA_REF'], dayfirst=True) #convertendo para data, nunca esquecer de passar dayfirst=True, pois senão troca dia e mes
base_df['ANOMES'] = base_df['DATA_REF'].dt.strftime('%Y%m') #criando uma coluna com o ano e mês
base_df = base_df.drop(columns=['DATA_REF','FX_ATRASO_INI'])

colunas = {
    'ATRASO_INI':'DIAS_ATRASO',
    'VL_LIQ_VENDA':'VLV',
    'DATA_PAGAMENTO': 'DT_SUC_BANCO',
    'CAIXA_PERF':'VLR_PERF_BANCO',
    'TIPO_SUCESSO':'TIPO_SUC_BANCO'
}
base_df.rename(columns=colunas, inplace=True)

#gerando um dataframe transposto das colunas de FLAGS
df_dummies = pd.get_dummies(base_df['MARCACAO'], prefix='FLAG', prefix_sep='_') # faz a tranposição, criando uma coluna para cada status
df_dummies = df_dummies.astype(int) #mudando o retorno de true e false para 0 e 1
base_df = pd.concat([base_df, df_dummies], axis=1) # juntando as colunas novas
base_df = base_df.drop(columns=['MARCACAO','FLAG_ESTOQUE']) #excluindo a coluna redundante

columns_to_fill = ['TIPO_SUC_BANCO','DT_SUC_BANCO', 'VLR_PERF_BANCO']
base_df[columns_to_fill] = base_df[columns_to_fill].fillna('')

base_df.columns

Index(['CTT_OP', 'VLCLSER', 'CARTEIRA', 'FASE', 'UF', 'VLV', 'GRUPO_PERF',
       'DIAS_ATRASO', 'TIPO_SUC_BANCO', 'DT_SUC_BANCO', 'VLR_PERF_BANCO',
       'DT_ATRASO_INI', 'META_CC', 'META_RET', 'FAIXA_BANCO', 'FAIXA_FIM',
       'ANOMES', 'FLAG_ENTRADA'],
      dtype='object')

In [ ]:
#importando os daddos de HC
colunas = ['LOGIN', 'STATUS', 'ESCOPO', 'SUPERVISOR', 'CLASS', 'ANOMES']

#encoding='utf-8-sig #se parar de funcionar tentar esse charset'
hc_df = pd.read_csv(
    input_path + 'hc.csv',
    encoding='latin-1',
    sep=';',
    low_memory=False,
    dtype = str,
    usecols = colunas
  )

#limpando textos indesejados
limpar_colunas(hc_df, colunas)

rename = {
  'SUPERVISOR':'GESTOR',
  'ESCOPO':'EQUIPE'
}
hc_df.rename(columns=rename, inplace=True)
hc_df.columns

Index(['LOGIN', 'STATUS', 'EQUIPE', 'GESTOR', 'CLASS', 'ANOMES'], dtype='object')

In [ ]:
#importando os sucessos
colunas = ['CTT_OP', 'PARC', 'DATA_SUC', 'H.O', 'VLRDESP', 'VLRPG', 'ESP', 'OPER_FIM', 'NRO_BOL',
'TP_SUC', 'FONTE', 'REPASSE', 'VLV', 'DATA_REF']

#encoding='utf-8-sig #se parar de funcionar tentar esse charset'
sucessos_df = pd.read_csv(
    input_path + 'sucessos.csv',
    encoding='latin-1',
    sep=';',
    low_memory=False,
    dtype = str,
    usecols = colunas
  )

sucessos_df.drop_duplicates(inplace=True) #excluindo duplicatas
rename = {
  'TP_SUC':'TIPO_SUCESSO',
  'OPER_FIM': 'LOGIN',
  'ESP': 'LOGIN_ESP',
  'H.O': 'HO',
  'NRO_BOL': 'NR_BOL',
  'VLRPG': 'VLR_PAGO',
  'VLRDESP': 'VLR_DESPESA'
}
sucessos_df.rename(columns=rename, inplace=True)

#limpando textos indesejados
colunas = ['CTT_OP','PARC','TIPO_SUCESSO','NR_BOL','LOGIN', 'LOGIN_ESP', 'VLR_DESPESA', 'VLR_PAGO','HO','REPASSE','VLV']
limpar_colunas(sucessos_df, colunas)

# Convertendo os tipos de dados
colunas = ['VLR_PAGO','VLR_DESPESA','HO','REPASSE','VLV']
to_numeric(sucessos_df, colunas)

# renomeando os tipos de sucesso, para poder fazer a classificação
sucessos_df['TIPO_SUCESSO'] = sucessos_df['TIPO_SUCESSO'].replace(
    {
    'apreensao': '01.BA',
    'entrega': '02.EA',
    'quitacao': '03.QUIT',
    'refin': '04.REFIN',
    'atualizacao': '05.ATUA',
    'parcial': '06.PARC'
  }
)

df_pagamento = sucessos_df[sucessos_df['PARC'] != ''] # Filtrar DataFrame com PARC não vazio
df_demais = sucessos_df[sucessos_df['PARC'] == ''] # Filtrar DataFrame refin,retomado

df_reduzido = df_pagamento.groupby('NR_BOL').agg({
    'CTT_OP': 'first',
    'PARC': lambda parcelas: ';'.join(str(p) for p in parcelas if pd.notna(p) and str(p) != ''),  # Convertendo para string
    'DATA_SUC': 'first',
    'HO': lambda ho: round(ho.sum(), 2),
    'VLR_DESPESA': 'sum',
    'VLR_PAGO': 'sum',
    'LOGIN': 'first',
    'LOGIN_ESP': 'first',
    'TIPO_SUCESSO': 'first',
    'FONTE': 'first',
    'REPASSE': lambda repasse: round(repasse.sum(), 2),
    'VLV': 'first',
    'DATA_REF': 'first',
}).reset_index()

df_reduzido = pd.concat([df_reduzido, df_demais], ignore_index=True) #juntando pagamentos e demais sucessos

#classificando os registros
df_sucessos = df_reduzido.sort_values(
    by=['CTT_OP', 'TIPO_SUCESSO','FONTE'],
    ascending=[True, True,True]
)

#df_sucessos.drop_duplicates(subset=['CTT_OP', 'TIPO_SUCESSO'], keep='first', inplace=True) #mantendo apenas o melhor sucesso do contrato, que vai para a performance
df_sucessos['TIPO_SUCESSO'] = df_sucessos['TIPO_SUCESSO'].str.slice(3) #extrai os caracteres a partir da posição 3, ou seja, do quarto em diante
df_sucessos['DATA_REF'] = pd.to_datetime(df_sucessos['DATA_REF'], dayfirst=True) #convertendo para data
df_sucessos['ANOMES'] = df_sucessos['DATA_REF'].dt.strftime('%Y%m') #criando uma coluna com o ano e mês
df_sucessos = df_sucessos.drop(columns=['DATA_REF'])

conditions = [
  df_sucessos['TIPO_SUCESSO'] == 'EA',
  df_sucessos['TIPO_SUCESSO'] == 'BA',
  df_sucessos['REPASSE'] >= 0
]

status = [
  df_sucessos['VLV'],
  df_sucessos['VLV'],
  df_sucessos['REPASSE']
]

df_sucessos['VLR_PERF'] = np.select(conditions, status, default=0)

#gerando um dataframe transposto das colunas de FLAGS
df_dummies = pd.get_dummies(df_sucessos['TIPO_SUCESSO'], prefix='FLAG', prefix_sep='_') # faz a tranposição, criando uma coluna para cada status
df_dummies = df_dummies.astype(int) #mudando o retorno de true e false para 0 e 1
df_sucessos = pd.concat([df_sucessos, df_dummies], axis=1) # juntando as colunas novas

df_sucessos['FLAG_RET'] = df_sucessos['FLAG_EA'] + df_sucessos['FLAG_BA']
df_sucessos['FLAG_PGTO'] = df_sucessos['FLAG_ATUA'] + df_sucessos['FLAG_PARC'] + df_sucessos['FLAG_QUIT']
df_sucessos['FLAG_SUC'] = df_sucessos['FLAG_RET'] + df_sucessos['FLAG_PGTO'] + df_sucessos['FLAG_REFIN']

#adicionando as colunas de HC
df_sucessos = pd.merge(df_sucessos, hc_df[['LOGIN', 'EQUIPE', 'GESTOR', 'CLASS', 'ANOMES']],
on=['LOGIN', 'ANOMES'], how='left', suffixes=('', '_NEW'))

df_sucessos.loc[df_sucessos['LOGIN'] == 'nage',['EQUIPE','CLASS']] = 'retomada_judicial'
columns_to_fill = ['EQUIPE','GESTOR', 'CLASS']
df_sucessos[columns_to_fill] = df_sucessos[columns_to_fill].fillna('outros')

df_sucessos['VLR_PERF'] = df_sucessos['VLR_PERF'].fillna(0)  # Preencher valores nulos com 0
df_sucessos['VLR_PERF'] = df_sucessos['VLR_PERF'].astype(float)  # Converter para float

#formatando a data de saída
df_sucessos['DATA_SUC'] = pd.to_datetime(df_sucessos['DATA_SUC'], dayfirst=True)
df_sucessos['DATA_SUC'] = df_sucessos['DATA_SUC'].dt.date

df_sucessos.columns

df_sucessos = pd.merge(
    df_sucessos,
    base_df[['CTT_OP', 'GRUPO_PERF', 'FASE', 'FAIXA_BANCO', 'FAIXA_FIM', 'ANOMES']],
    on=['CTT_OP', 'ANOMES'],
    how='left',
    suffixes=('', '_INTERNO')
)

df_sucessos['FLAG_PERF'] = np.where((df_sucessos['FLAG_SUC'] > 0) & (df_sucessos['GRUPO_PERF'] == 'VEIC'), 1, 0) #definindo quais linhas irão performar

df_sucessos.sort_values(by=['ANOMES','CTT_OP'], inplace=True) #classificando os dados, para agrupar os anomes
ordenar_colunas = [
  'CTT_OP', 'PARC', 'GRUPO_PERF', 'FASE', 'FAIXA_BANCO', 'FAIXA_FIM', 'LOGIN', 'LOGIN_ESP', 'DATA_SUC',
  'NR_BOL', 'VLR_PAGO', 'VLR_DESPESA', 'HO', 'TIPO_SUCESSO', 'FONTE', 'GESTOR','EQUIPE', 'CLASS', 'REPASSE',
  'VLV','VLR_PERF', 'FLAG_PERF', 'FLAG_REFIN', 'FLAG_RET', 'FLAG_BA', 'FLAG_EA', 'FLAG_PGTO','FLAG_QUIT','FLAG_ATUA', 'FLAG_PARC',
  'FLAG_SUC', 'ANOMES'
]
df_sucessos = df_sucessos[ordenar_colunas]

columns_to_fill = ['GRUPO_PERF','FASE', 'FAIXA_FIM']
df_sucessos[columns_to_fill] = df_sucessos[columns_to_fill].fillna('BASE_OUT')

#df_sucessos_columns = list(df_sucessos.columns)
display(df_sucessos.columns)

Index(['CTT_OP', 'PARC', 'GRUPO_PERF', 'FASE', 'FAIXA_BANCO', 'FAIXA_FIM',
       'LOGIN', 'LOGIN_ESP', 'DATA_SUC', 'NR_BOL', 'VLR_PAGO', 'VLR_DESPESA',
       'HO', 'TIPO_SUCESSO', 'FONTE', 'GESTOR', 'EQUIPE', 'CLASS', 'REPASSE',
       'VLV', 'VLR_PERF', 'FLAG_PERF', 'FLAG_REFIN', 'FLAG_RET', 'FLAG_BA',
       'FLAG_EA', 'FLAG_PGTO', 'FLAG_QUIT', 'FLAG_ATUA', 'FLAG_PARC',
       'FLAG_SUC', 'ANOMES'],
      dtype='object')

In [ ]:
########################## CRIANDO O OUTPUT DE SUCESSOS [sucessos_2024.csv] ##########################

#trocando os pontos por virgula para deixar já no formato de exportação excel correto
output_sucessos = df_sucessos.map(lambda x: str(x).replace('.', ','))
output_sucessos[['FAIXA_BANCO', 'FAIXA_FIM']] = output_sucessos[['FAIXA_BANCO', 'FAIXA_FIM']].apply(lambda x: x.str.replace(',', '.')) #corrigindo as faixas

#definindo colunas de saida
output_columns = [
    'CTT_OP', 'PARC', 'GRUPO_PERF', 'FASE', 'FAIXA_BANCO', 'FAIXA_FIM', 'LOGIN', 'LOGIN_ESP', 'DATA_SUC',
    'NR_BOL', 'VLR_PAGO', 'VLR_DESPESA', 'HO', 'TIPO_SUCESSO', 'FONTE', 'GESTOR','EQUIPE', 'CLASS', 'REPASSE',
    'VLV','VLR_PERF', 'FLAG_PERF', 'FLAG_REFIN', 'FLAG_RET', 'FLAG_BA', 'FLAG_EA', 'FLAG_PGTO','FLAG_QUIT','FLAG_ATUA', 'FLAG_PARC',
      'FLAG_SUC', 'ANOMES'
]

output_sucessos = output_sucessos[output_columns]
export_to_csv(output_sucessos, output_path,file_name='sucessos_2024.csv',index=False)
output_sucessos.columns

Index(['CTT_OP', 'PARC', 'GRUPO_PERF', 'FASE', 'FAIXA_BANCO', 'FAIXA_FIM',
       'LOGIN', 'LOGIN_ESP', 'DATA_SUC', 'NR_BOL', 'VLR_PAGO', 'VLR_DESPESA',
       'HO', 'TIPO_SUCESSO', 'FONTE', 'GESTOR', 'EQUIPE', 'CLASS', 'REPASSE',
       'VLV', 'VLR_PERF', 'FLAG_PERF', 'FLAG_REFIN', 'FLAG_RET', 'FLAG_BA',
       'FLAG_EA', 'FLAG_PGTO', 'FLAG_QUIT', 'FLAG_ATUA', 'FLAG_PARC',
       'FLAG_SUC', 'ANOMES'],
      dtype='object')

In [ ]:
########################## GERANDO AS COLUNAS DE SUCESSO ESCOB PARA COMPARAR COM OS DADOS DO BANCO ##########################

colunas_necessarias = ['CTT_OP','TIPO_SUCESSO','VLR_PERF', 'FLAG_PERF', 'ANOMES']
sucesso_resumido = df_sucessos[colunas_necessarias]

colunas = [
    'CTT_OP', 'GRUPO_PERF', 'CARTEIRA', 'FASE', 'UF', 'FLAG_ENTRADA', 'DT_ATRASO_INI', 'DIAS_ATRASO', 'FAIXA_BANCO', 'FAIXA_FIM', 'VLCLSER','META_CC', 'META_RET',
    'VLV','TIPO_SUC_BANCO', 'DT_SUC_BANCO', 'VLR_PERF_BANCO',  'ANOMES'
]  # Colunas que você deseja manter
base_final = base_df[colunas]

filtro_tp_suc = sucesso_resumido.drop_duplicates(subset=['CTT_OP', 'ANOMES'], keep='first')
df_ret = filtro_tp_suc[['CTT_OP','TIPO_SUCESSO', 'ANOMES']]
df_ret.rename(columns={'TIPO_SUCESSO': 'TP_SUC_INTERNO'}, inplace=True)

base_final = pd.merge(
    base_final,
    df_ret,
    left_on=['CTT_OP', 'ANOMES'],
    right_on=['CTT_OP', 'ANOMES'],
    how='left',
)

#breakpoint()
filtro_pagto = sucesso_resumido[~sucesso_resumido['TIPO_SUCESSO'].isin(['EA', 'BA', 'REFIN'])]
df_ret = round(filtro_pagto.groupby(['CTT_OP', 'ANOMES'])['VLR_PERF'].sum().reset_index(),2)
df_ret.rename(columns={'VLR_PERF': 'PERF_PGTO_ESCOB'}, inplace=True)

base_final = pd.merge(
    base_final,
    df_ret[['CTT_OP', 'ANOMES','PERF_PGTO_ESCOB']],
    left_on=['CTT_OP', 'ANOMES'],
    right_on=['CTT_OP', 'ANOMES'],
    how='left',
)

filtro_ret = sucesso_resumido[sucesso_resumido['TIPO_SUCESSO'].isin(['EA', 'BA'])]
df_ret = round(filtro_ret.groupby(['CTT_OP', 'ANOMES'])['VLR_PERF'].sum().reset_index(),2)
df_ret.rename(columns={'VLR_PERF': 'PERF_RET_ESCOB'}, inplace=True)

base_final = pd.merge(
    base_final,
    df_ret[['CTT_OP', 'ANOMES','PERF_RET_ESCOB']],
    left_on=['CTT_OP', 'ANOMES'],
    right_on=['CTT_OP', 'ANOMES'],
    how='left',
)

filtro_refin = sucesso_resumido[sucesso_resumido['TIPO_SUCESSO'].isin(['REFIN'])]
df_ret = round(filtro_refin.groupby(['CTT_OP', 'ANOMES'])['VLR_PERF'].count().reset_index(),2)
df_ret.rename(columns={'VLR_PERF': 'PERF_REFIN_ESCOB'}, inplace=True)

base_final = pd.merge(
    base_final,
    df_ret[['CTT_OP', 'ANOMES','PERF_REFIN_ESCOB']],
    left_on=['CTT_OP', 'ANOMES'],
    right_on=['CTT_OP', 'ANOMES'],
    how='left',
)

base_final['PERF_INTERNA'] = np.where(
  base_final['PERF_RET_ESCOB']>0,
  base_final['PERF_RET_ESCOB'],
  np.where(
    base_final['PERF_PGTO_ESCOB'] > 0,
    base_final['PERF_PGTO_ESCOB'],
    0
  )
)

base_final['FLAG_PERF'] = np.where((base_final['PERF_REFIN_ESCOB'] > 0) | (base_final['PERF_INTERNA'] > 0), 1, 0) #definindo quais linhas irão performar

#export_to_csv(base_final, output_path)
base_final.columns

<ipython-input-8-03be401745ba>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ret.rename(columns={'TIPO_SUCESSO': 'TP_SUC_INTERNO'}, inplace=True)


Index(['CTT_OP', 'GRUPO_PERF', 'CARTEIRA', 'FASE', 'UF', 'FLAG_ENTRADA',
       'DT_ATRASO_INI', 'DIAS_ATRASO', 'FAIXA_BANCO', 'FAIXA_FIM', 'VLCLSER',
       'META_CC', 'META_RET', 'VLV', 'TIPO_SUC_BANCO', 'DT_SUC_BANCO',
       'VLR_PERF_BANCO', 'ANOMES', 'TP_SUC_INTERNO', 'PERF_PGTO_ESCOB',
       'PERF_RET_ESCOB', 'PERF_REFIN_ESCOB', 'PERF_INTERNA', 'FLAG_PERF'],
      dtype='object')

In [ ]:
########################## GERANDO O OUTPUT DA BASE FINAL [ BASE + SUCESSO_ESCOB ] ##########################

output_columns = [
  'CTT_OP', 'GRUPO_PERF', 'CARTEIRA', 'FASE', 'UF', 'DT_ATRASO_INI', 'DIAS_ATRASO', 'FAIXA_BANCO', 'FAIXA_FIM',
  'VLCLSER',  'META_CC', 'META_RET',  'VLV','TIPO_SUC_BANCO', 'DT_SUC_BANCO', 'VLR_PERF_BANCO', 'FLAG_ENTRADA',
  'TP_SUC_INTERNO', 'PERF_INTERNA', 'PERF_PGTO_ESCOB', 'PERF_RET_ESCOB', 'PERF_REFIN_ESCOB', 'FLAG_PERF', 'ANOMES'
]  # Colunas que você deseja manter
output_base = base_final[output_columns] #reordenando as colunas

# Convertendo os tipos de dados
colunas = ['PERF_INTERNA','PERF_PGTO_ESCOB', 'PERF_RET_ESCOB', 'PERF_REFIN_ESCOB']
to_numeric(output_base, colunas)
output_base = output_base.sort_values(by=['ANOMES', 'PERF_INTERNA'], ascending=[False, False])

output_base = output_base.map(lambda x: str(x).replace('.', ',').replace('nan', ''))
output_base[['FAIXA_BANCO', 'FAIXA_FIM']] = output_base[['FAIXA_BANCO', 'FAIXA_FIM']].apply(lambda x: x.str.replace(',', '.')) #corrigindo as faixas
export_to_csv(output_base, output_path,file_name='base_final_2024.csv',index=False)

In [ ]:
to_numeric(base_df, ['CAIXA_PERF'])
to_numeric(base_df, ['VLR_PERF'])
caixa_banco = round(base_df['CAIXA_PERF']/100, 0) * 100
caixa_interno = round(base_df['VLR_PERF']/100, 0) * 100

conditions = [
  caixa_banco == caixa_interno,
  caixa_banco > caixa_interno,
  caixa_banco < caixa_interno,
  pd.isna(base_df['TIPO_SUCESSO'])
]

status = [
  'IGUAL',
  'MAIOR',
  'MENOR',
  'FALTANDO PERF'
]
base_df['ST_PERF'] = np.select(conditions, status, default='FALTANDO PERF')

to_numeric(base_df, ['VLR_REPASSE']) # elimna também os nan
base_df['VLR_REPASSE'] = base_df['VLR_REPASSE'].astype(str).str.replace('.', ',', regex=False)
#display(base_df.head())
#base_df = base_df.head(10000)


KeyError: 'CAIXA_PERF'

In [ ]:
# Criar a tabela de resumo
tabela_resumo = resumo_suc.pivot_table(
    values='VLR_PERF',
    index='EQUIPE',
    columns='ANOMES',
    aggfunc='sum',
    fill_value=0
)

tabela_resumo = tabela_resumo.apply(lambda x: x / x.sum() * 100, axis=0) # Calcular a porcentagem de sucesso para cada equipe e mês
tabela_resumo['Total'] = tabela_resumo.sum(axis=1) / tabela_resumo.shape[1]
tabela_resumo.fillna(0, inplace=True)
tabela_resumo = tabela_resumo.map(lambda x: f"{x:.2f}%")
tabela_resumo.index.name = 'EQUIPE'
tabela_resumo = tabela_resumo.map(lambda x: str(x).replace('.', ','))

#export_to_csv(tabela_resumo, output_path,file_name='equipe_vs_ano.csv',index=True)

In [ ]:
#cria flags comparando duas tabelas
def flagCreator(col_flag_name, main_df, df_to_filter, merge_columns, lambda_conditions=None):
    """Cria uma coluna de flag binária em main_df com base em uma condição de filtro em df_to_filter.
    Args:
        col_flag_name (str): Nome da coluna de flag a ser criada em main_df.
        main_df (pd.DataFrame): DataFrame principal onde a flag será criada.
        df_to_filter (pd.DataFrame): DataFrame a ser filtrado para encontrar correspondências.
        merge_columns (list): Lista de colunas a serem usadas para o merge.
        lambda_conditions (function, optional): Função lambda que retorna True para as linhas que devem ser incluídas no filtro.
            Se não for fornecida, todas as linhas de df_to_filter serão usadas. Defaults to None.
    Returns:
        pd.DataFrame: DataFrame main_df com a coluna de flag adicionada.
    """

    main_df[col_flag_name] = 0  # Inicializa a coluna com 0

    # Se lambda_conditions não for fornecido, use todas as linhas de df_to_filter
    if lambda_conditions is None:
        filtered_df = df_to_filter[merge_columns]
    else:
        filtered_df = df_to_filter[lambda_conditions(df_to_filter)][merge_columns]

    base_df = main_df.merge(
        filtered_df,
        on=merge_columns,
        left_on=['CTT_OP', 'DATA_REF'],
        right_on=['CTT_OP', 'DATA_REF'],
        how='left',
        indicator=True
    )
    main_df.loc[base_df['_merge'] == 'both', col_flag_name] = 1  # Atualiza a flag para 1 onde houve correspondência
    return main_df

bases_df = flagCreator( 'FLAG_SUC', bases_df, sucessos_df, ['CTT_OP', 'DATA_REF']) # Cria FLAG_SUC
bases_df = flagCreator('FLAG_EA', bases_df, sucessos_df, ['CTT_OP', 'DATA_REF'], lambda sucessos_df: sucessos_df['TP_SUC'] == 'Entrega') # Cria FLAG_EA
bases_df = flagCreator( 'FLAG_BA', bases_df, sucessos_df, ['CTT_OP', 'DATA_REF'], lambda sucessos_df: sucessos_df['TP_SUC'] == 'Apreensão') # Cria FLAG_BA
bases_df.to_csv('/content/drive/MyDrive/bases_df.csv', index=False, sep=';', encoding='utf-8-sig')
bases_df.columns

In [ ]:
#bases_df[['CTT_OP','']]

In [ ]:
def comentar():

  bases_df['FLAG_SUC'] = 0 # Adiciona a coluna 'FLAG_SUC' inicialmente com valor 0
  base_df_suc = bases_df.merge(sucessos_df[['CTT_OP', 'DATA_REF']], left_on=['CTT_OP', 'DATA_REF'], how='left', indicator=True)
  # Atualiza a coluna FLAG_SUC para 1 onde houve correspondência
  bases_df['FLAG_SUC'] = (base_df_suc['_merge'] == 'both').astype(int)  #cria uma coluna chamada '_merge' com dois retornos possíveis, left_only e both

  bases_df['FLAG_EA'] = 0 # Adiciona a coluna 'FLAG_EA' inicialmente com valor 0
  base_df_ea = bases_df.merge(
      sucessos_df[sucessos_df['TP_SUC'] == 'Entrega'][['CTT_OP', 'DATA_REF']], #filtra as linhas = entregas, filtra as colunas 'CTT_OP' e 'DATA_REF'
      on=['CTT_OP', 'DATA_REF'],
      how='left',
      indicator=True
  )
  bases_df.loc[base_df_ea['_merge'] == 'both', 'FLAG_EA'] = 1 # Atualiza a coluna FLAG_EA para 1 onde houve correspondência com TP_SUC == 'Entrega'

  bases_df['FLAG_BA'] = 0 # Adiciona a coluna 'FLAG_EA' inicialmente com valor 0
  base_df_ea = bases_df.merge(sucessos_df[sucessos_df['TP_SUC'] == 'Apreensão'][['CTT_OP', 'DATA_REF']], on=['CTT_OP', 'DATA_REF'], how='left', indicator=True)
  bases_df.loc[base_df_ea['_merge'] == 'both', 'FLAG_BA'] = 1 # Atualiza a coluna FLAG_EA para 1 onde houve correspondência com TP_SUC == 'Entrega'



In [ ]:
agrupado = bases_df.groupby(['DATA_REF', 'TIPO_SUC_FIM'])['FLAG_SUC'].sum().unstack()
agrupado['TOTAL'] = agrupado.sum(axis=1)
display(agrupado)

In [ ]:
#sucessos_df = pd.merge(sucessos_df, bases_df[['CTT_OP', 'DATA_REF', 'FASE']], on=['CTT_OP', 'DATA_REF'], how='left') #adicionando a coluna de fase
#sucessos_df.to_csv('/content/drive/MyDrive/sucessos_com_fase.csv', index=False, encoding='utf-8-sig')

In [ ]:
def amostragem_estratificada(df, group_columns, perc_member):
  """
  Realiza amostragem estratificada em um DataFrame com base em múltiplos níveis de agrupamento.
  Args:
      df (pd.DataFrame): DataFrame de entrada.
      group_columns (list): Lista de nomes de colunas para agrupar.
      perc_member (float): Porcentagem de elementos a serem amostrados dentro de cada grupo.

  Returns:
      pd.DataFrame: DataFrame com a amostra estratificada.
  """
  # Aplicar a amostragem aleatória em cada grupo
  aux = df.groupby(group_columns, group_keys=False).apply(lambda x: x.sample(frac=perc_member))
  return aux

#amostra = amostragem_estratificada(sucessos_df,['DATA_REF','FAIXA_ATRASO'],0.5)
#amostra.describe()